<a href="https://www.kaggle.com/code/hmelmoth/midi-generation?scriptVersionId=217926923" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Setup

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.image_segmentation.git
from gcpds.image_segmentation.losses import DiceCoefficient

  Cloning https://github.com/UN-GCPDS/python-gcpds.image_segmentation.git to /tmp/pip-req-build-q7lx6zug
  Running command git clone --filter=blob:none --quiet https://github.com/UN-GCPDS/python-gcpds.image_segmentation.git /tmp/pip-req-build-q7lx6zug
  Resolved https://github.com/UN-GCPDS/python-gcpds.image_segmentation.git to commit 01b06059d3a0d2980eea4d0b1e964718cab173ee
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for gdown from https://files.pythonhosted.org/packages/54/70/e07c381e6488a77094f04c85c9caf1c8008cdc30778f7019bc52e5285ef0/gdown-5.2.0-py3-none-any.whl.metadata
  Created wheel for gcpds-image-segmentation: filename=gcpds_image_segmentation-0.1a0-py3-none-any.whl size=39399 sha256=d7420e395ee6ee89b47addcffb068f86ecdc16769e7a24aeafa5e8ab24c48281
  Stored in directory: /tmp/pip-ephem-wheel-cache-87xoz20s/wheels/88/5f/53/179661a69fb02b7c932e1f3bff4d7255dc2bcbe9a72c883761
Successfully built gcpds-image-segmentation


In [3]:
#!pip install tensorflow==2.8

In [4]:
#!pip install scikeras[tensorflow] >/dev/null

In [5]:
!git clone https://github.com/hdperezn/MIDI_functions.git
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth
!pip install pretty_midi

Cloning into 'MIDI_functions'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 185 (delta 77), reused 71 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (185/185), 8.40 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libdouble-conversion3 libegl-mesa0 libegl1 libevdev2
  libfluidsynth2 libgbm1 libgudev-1.0-0 libinput-bin libinput10
  libinstpatch-1.0-2 libmtdev1 libpcre2-16-0 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom2 libwayland-server0 libxcb-icccm4 libxcb-image0
  libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0
  libxcb-xinput0 libxcb-xkb1 libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme
  qttr

In [6]:
import os
import math
import IPython
import string
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt


import fluidsynth
import glob
import pathlib
import pretty_midi

from IPython import display
from scipy.spatial.distance import cdist
from MIDI_functions.functions.MIDI_functions import cut_midi_secTrial,piano_roll_to_pretty_midi
from MIDI_functions.functions.UNetLike_PianoRolls import UNet_Pianoroll, Autoencoder_Pianoroll


from tensorflow.keras.regularizers import l1_l2
from sklearn.model_selection import StratifiedShuffleSplit
#from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, ConfusionMatrixDisplay


# some usefull functions (window EEG and custom loss) and paths

In [11]:
#Paths
path_DEAP = '/kaggle/input/deap-dataset'
#save_path = '/content/drive/MyDrive/music_VAEs/DEAP_dataset/'
video_list = pd.read_excel("/kaggle/input/d/manh123df/deap-dataset/deap-dataset/Metadata/video_list_fixed.xlsx") #Metadata/video_list_fixed.xlsx
video_list_experiment = video_list.dropna(subset= ['Experiment_id'])
# path to load EEGnet weigths
EEGnet_weigths_path = '/content/drive/MyDrive/music_VAEs/DEAP_dataset/EEG/'

## trials out
trials_toCUT_inEEG = np.asarray([1, 18,19, 22,23, 27, 33, 43, 79])

In [12]:
def segmentation_trials(X, fs, segs= 6):
  """
  Input
  X: numpy array with EEG info, dims (trials,channels,time)
  fs: int value with the sampling frequency
  segs: int vale with the length of the trial

  out
  X_train: numpy array with the eeg data windowed in trials with length "segs"
  """

  X_train = []
  windows = int(X.shape[2]/(segs*fs))
  for i in range(X.shape[0]):
    for j in range(windows):
        #print(j*(6*fs),int((j+1)*(segs*fs)))
        X_train.append(X[i,:,j*(segs*fs):int((j+1)*(segs*fs))])
  return np.asarray(X_train)

In [13]:
def pading_midis(Prolls_windowed, Prolls_trial_len = 128):
  """
  function to cut or padding the midi arrays to equal lengs
  """
  Prolls_cut = []
  for i in range(len(Prolls_windowed)):
    if Prolls_windowed[i].shape[2] >= Prolls_trial_len:
      x = Prolls_windowed[i][:,:,0:Prolls_trial_len]
      Prolls_cut.append(x)
    else:
      dif = Prolls_trial_len - Prolls_windowed[i].shape[2]
      x = np.concatenate( (Prolls_windowed[i][:,:,:],Prolls_windowed[i][:,:,-dif::] ), axis = 2)
      Prolls_cut.append(x)
  return np.concatenate(np.array(Prolls_cut, dtype=object), axis = 0)


In [14]:
def pearson_multidims(y_true, y_pred ):
  num = tf.linalg.trace(tf.linalg.matmul(y_true,y_pred, transpose_a=True))
  # -------------------------
  dem1 = tf.linalg.trace(tf.linalg.matmul(y_true,y_true, transpose_a=True))
  dem2 = tf.linalg.trace(tf.linalg.matmul(y_pred,y_pred, transpose_a=True))
  dem = tf.sqrt(dem1)*tf.sqrt(dem2)
  return num/dem

def custom_lossRho(y_true, y_pred):
  # -------------------------Pearson loss----------------------------
  mean_point=y_pred.shape[-1]//2
  phyEEG=tf.cast(y_pred[:,0:mean_point], tf.float32)
  phyenco=tf.cast(y_pred[:,mean_point:], tf.float32)
  return pearson_multidims(phyEEG, phyenco)

In [15]:
def split_EEGNet(eegnet, layer_split = 11):
  # ----------- encoding layers
  imput_encoding = tf.keras.layers.Input(shape = (Chans, Samples, 1))
  layer = imput_encoding
  for i in range(1,layer_split):
      layer = eegnet.layers[i](layer)
  EEGNet_encoding_layer = tf.keras.models.Model(inputs=imput_encoding, outputs=layer)
  EEGNet_encoding_layer.trainable == False
  # ----------- clasification layers
  imput_clasification = tf.keras.layers.Input(shape = layer.shape[1::])
  layer = imput_clasification
  for i in range(layer_split,len(eegnet.layers)):
      layer = eegnet.layers[i](layer)
  EEGNet_clasification_layer = tf.keras.models.Model(inputs=imput_clasification, outputs=layer)
  EEGNet_clasification_layer.trainable == False

  return EEGNet_encoding_layer, EEGNet_clasification_layer

In [16]:
def rebuilPianoRolls(X):
  """
  this function takes an output array from the vae net and returns
  a piano roll array ready to be converted into MIDI file
  input
  X: array with dimensions [samples, time, pitch, 1]
  output
  proll_padding: array with dimension [sample x pitch x time ]
  """
  #trasnpose to [sample x time x pitch]
  proll = X[:,:,:,0].transpose(0,2,1)
  proll_padding =np.pad(proll, ((0,0), (24,40), (0, 0)), 'constant')
  return proll_padding

# Create Piano roll train, test files

In [17]:
path_midi = '/kaggle/working/MIDI_functions/deap_midis'
data_dir = pathlib.Path(path_midi)
filenames = glob.glob(str(data_dir/'*.mid*'))
print('Number of files:', len(filenames))

Number of files: 40


In [18]:
_SAMPLING_RATE = 16000
fProll = 20

In [19]:
Prolls_windowed_train = []
for i in range(len(filenames)):
  #load MIDI and create array
  Proll = pretty_midi.PrettyMIDI(filenames[i]).get_piano_roll(fs=fProll)
  if i in [0, 13, 16, 21, 39]:
    Proll_cut = cut_midi_secTrial(Proll, 9)
  elif i in [9, 11]:
    Proll_cut = cut_midi_secTrial(Proll, 8)
  else:
    Proll_cut = cut_midi_secTrial(Proll, 10)
  Prolls_windowed_train.append(Proll_cut[:])

Prolls_array_train = pading_midis(Prolls_windowed_train, Prolls_trial_len = 128)
Prolls_train_mask = np.where(Prolls_array_train>0,1,0)

print('these are de dims of the MIDI data to use in the neural network: ')
print('     train data:', Prolls_array_train.shape)


these are de dims of the MIDI data to use in the neural network: 
     train data: (391, 128, 128)


In [20]:
indx_full_audio = []#np.zeros(len(filenames))
all_trials = np.array(range(400))
for i in range(len(filenames)):
  #load MIDI and create array
  if i in [0, 13, 16, 21, 39]:
    indx_full_audio.extend([1,1,1,1,1,1,1,1,1,np.nan])
  elif i in [9, 11]:
    indx_full_audio.extend([1,1,1,1,1,1,1,1,np.nan,np.nan])
  else:
    indx_full_audio.extend([1,1,1,1,1,1,1,1,1,1])
indx_full_audio = all_trials*np.array(indx_full_audio)

indx_full_audio = indx_full_audio[~np.isnan(indx_full_audio)].astype(int)


In [21]:
# save_models_path = '/content/drive/MyDrive/music_VAEs/DEAP_dataset/EEG/cross_val_results/'
# mask_probs = np.where(np.load(save_models_path+'top_20_mask_probas.npy') > 0.5, 1, 0)

# Prolls_windowed_train = []
# for i in range(len(filenames)):
#   #load MIDI and create array
#   mask_song = mask_probs[i]

#   Proll = pretty_midi.PrettyMIDI(filenames[i]).get_piano_roll(fs=fProll)
#   if i in [0, 13, 16, 21, 39]:
#     mask = mask_song[0:9] == 1
#     Proll_cut = cut_midi_secTrial(Proll, 9)[mask]
#   elif i in [9, 11]:
#     mask = mask_song[0:8] == 1
#     Proll_cut = cut_midi_secTrial(Proll, 8)[mask]
#   else:
#     mask = mask_song == 1
#     Proll_cut = cut_midi_secTrial(Proll, 10)[mask]

#   Prolls_windowed_train.append(Proll_cut[:])

# Prolls_array_train = pading_midis(Prolls_windowed_train, Prolls_trial_len = 128)
# Prolls_train_mask = np.where(Prolls_array_train>0,1,0)

# print('these are de dims of the MIDI data to use in the neural network: ')
# print('     train data:', Prolls_array_train.shape)

# #train_test_data
# Xpr_train = Prolls_array_train[:,24:88,:,np.newaxis].astype('float32').transpose(0,2,1,3)
# ypr_train = Prolls_train_mask[:,24:88,:,np.newaxis].astype('float32').transpose(0,2,1,3)
# Xpr_dims = Xpr_train.shape
# print('EEG variables names: "X_train"', Xpr_train.shape )
# print('MIDI variables names: "Xpr_train"')


# indx_full_audio = []#np.zeros(len(filenames))
# all_trials = np.array(range(400))
# for i in range(len(filenames)):
#   mask_song = mask_probs[i]
#   #load MIDI and create array
#   if i in [0, 13, 16, 21, 39]:
#     list_mask = np.array([1,1,1,1,1,1,1,1,1,np.nan])
#     idx_mask = np.where(mask_song == 0, np.nan, list_mask)

#     indx_full_audio.extend(idx_mask)

#   elif i in [9, 11]:
#     list_mask = np.array([1,1,1,1,1,1,1,1,np.nan,np.nan])
#     idx_mask = np.where(mask_song == 0, np.nan, list_mask)

#     indx_full_audio.extend(idx_mask)
#   else:
#     list_mask = np.array([1,1,1,1,1,1,1,1,1,1])
#     idx_mask = np.where(mask_song == 0, np.nan, list_mask)
#     indx_full_audio.extend(idx_mask)

# indx_full_audio = all_trials*np.array(indx_full_audio)

# indx_full_audio = indx_full_audio[~np.isnan(indx_full_audio)].astype(int)


# Functions

In [22]:
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l1_l2
from sklearn.base import  BaseEstimator, TransformerMixin, ClassifierMixin
import tensorflow_probability as tfp
class autoencoderCKA_Pianoroll(BaseEstimator, ClassifierMixin):
  def __init__(self, img_size, loss, num_classes = 1, labels = 2, epochs=50,batch_size=32, Q1= 20, Q2=2,
               learning_rate=1e-3,validation_split=0.2,verbose=1, droprate = 0.5, filters_list = [128, 64, 32],
               l1_l2 = 0,  Dice_weigths = [0.5,0.5],  plot_loss= True):

    self.img_size = img_size
    self.labels = labels
    self.loss = loss #must be two losses
    self.num_classes = num_classes
    self.epochs = epochs
    self.batch_size = batch_size
    self.learning_rate=learning_rate
    self.validation_split = validation_split
    self.verbose = verbose
    self.droprate = droprate
    self.plot_loss = plot_loss
    self.filters_list = filters_list
    self.Dice_weigths = Dice_weigths
    self.l1_l2 = l1_l2

    self.Q1 = Q1
    self.Q2 = Q2

  def Encoder(self, img_size ):
    self.encoder_inputs = tf.keras.Input(shape=(img_size))
    ### [First half of the network: downsampling inputs] ###
    filters = self.filters_list[::-1]

    # Entry block
    x = layers.Conv2D(filters[0], 3, strides=2, padding="same", name = 'conv_filter32_1')(self.encoder_inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for fil in filters[1::]:
        x = layers.Dropout(self.droprate)(x)
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(fil, 3, padding="same", name= 'Enco_conv_filter'+str(fil)+'_2')(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(fil, 3, padding="same", name= 'Enco_conv_filter'+str(fil)+'_3',
                                   kernel_regularizer=tf.keras.regularizers.L1L2(l1=self.l1_l2, l2=self.l1_l2))(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(fil, 1, strides=2, padding="same", name= 'Enco_conv_filter'+str(fil)+'_4'
        , kernel_regularizer=tf.keras.regularizers.L1L2(l1=self.l1_l2, l2=self.l1_l2))(previous_block_activation )
        x = layers.add([x, residual], name = 'Enco_add_block_filter'+str(fil))  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(fil, 3, strides= 2, padding="same", name= 'Enco_conv_filter'+str(fil)+'_5',
                                   kernel_regularizer=tf.keras.regularizers.L1L2(l1=self.l1_l2, l2=self.l1_l2))(x)
    self.x_nonflat = x
    x = layers.Flatten()(x)
    self.x = x
    self.encoder = tf.keras.Model(self.encoder_inputs, outputs = [self.x], name="encoder")
    return self.encoder

  def Decoder(self, num_classes):
    input_x = tf.keras.Input(shape=(int(self.x.shape[1])))

    #x = layers.Dense(int(8* 4* 128), activation='selu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=self.l1_l2,l2=self.l1_l2))(input_x)
    #x = input_x
    x = layers.Reshape((8, 4, 128))(input_x)

    x = layers.Conv2DTranspose(128, 3, strides= 2, padding="same", name= 'Deco_conv_filter'+str(128)+'_0',
                            kernel_regularizer=tf.keras.regularizers.L1L2(l1=self.l1_l2, l2=self.l1_l2))(x)
    previous_block_activation = x

    for filters in self.filters_list:
        x = layers.Dropout(self.droprate)(x)
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same", name= 'Deco_conv_filter'+str(filters)+'_1')(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same", name= 'Deco_conv_filter'+str(filters)+'_2',
                                   kernel_regularizer=tf.keras.regularizers.L1L2(l1=self.l1_l2, l2=self.l1_l2))(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same", name= 'Deco_conv_filter'+str(filters)+'_3',
                                 kernel_regularizer=tf.keras.regularizers.L1L2(l1=self.l1_l2, l2=self.l1_l2))(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    self.outputs = layers.Conv2D(num_classes, 3, activation="sigmoid", padding="same")(x)
    self.decoder = tf.keras.Model(inputs= [input_x], outputs = self.outputs, name="Decoder")
    return self.decoder

  def CKA_layers(self):
    regularizer_o = tf.keras.regularizers.OrthogonalRegularizer(factor=self.l1_l2)
    input_x = tf.keras.Input(shape=(self.x.shape[1]))

    #flattenA = tf.keras.layers.Flatten()(input_x)
    flattenA = layers.BatchNormalization()(input_x)

    h1A = layers.Dropout(self.droprate)(flattenA)
    h1A = tf.keras.layers.Dense(self.Q1,activation='selu',name='h1A',
                                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=self.l1_l2,l2=self.l1_l2))(h1A)
    h1A = layers.BatchNormalization()(h1A)
    h2A = tf.keras.layers.Dense(self.Q2,activation='linear',name='h2A',
                                kernel_regularizer=regularizer_o)(h1A)
    h2A = layers.BatchNormalization()(h2A)

    self.model = tf.keras.Model(inputs=[input_x],outputs=[h2A], name = 'cka_PianoRoll')
    return self.model


  def get_model(self, *_):
    seed = 123
    tf.random.set_seed(seed)
    np.random.seed(seed)
    tf.keras.backend.clear_session()

    self.winitializer = tf.keras.initializers.GlorotNormal(seed=seed)
    self.binitializer = "zeros"
    # ---- call layers -----
    enco = self.Encoder(self.img_size)
    deco = self.Decoder(self.num_classes)
    cka = self.CKA_layers()
    # ---- def red ---------
    block = enco(self.encoder_inputs)
    decoder_ = deco(block)
    cka_out = cka(block)


    # ----- MODEL -------

    metris = [tf.keras.metrics.Recall(), tf.keras.metrics.SpecificityAtSensitivity(0.1)]

    self.model = tf.keras.Model(inputs=[self.encoder_inputs],  outputs=[decoder_,cka_out], name = 'AE_CKA_MIDI')
    opt = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
    #binady cross entropy
    #loss_classification = tf.keras.losses.BinaryCrossentropy()
    self.model.compile(loss= self.loss , loss_weights = self.Dice_weigths,  optimizer= opt, metrics=metris)
    return

  def fit(self, X, Y, *_):
    # Y must be a list with the MIDI and the label
    callback = tf.keras.callbacks.TerminateOnNaN()

    self.get_model()
    self.history = self.model.fit(X, Y , epochs=self.epochs, batch_size=self.batch_size, validation_split=self.validation_split,
                                  callbacks=[callback],
                                  verbose=self.verbose)
    # ----- plot loss -----
    if self.plot_loss:
          self.plt_history()

  def predict(self, X,  *_):
    return self.model.predict(X)
  def plt_history(self):
    plt.plot(self.history.history['loss'])
    plt.plot(self.history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    return


class CKA_Pianoroll(BaseEstimator, ClassifierMixin):
  def __init__(self, img_size, loss, piano_roll_model , CNNf = 4, Q1= 20, Q2=2, l1_l2 = 1e-3,
               epochs=50, batch_size=32, learning_rate=1e-3, validation_split=0.2,
               verbose=1, droprate = 0.5, plot_loss= True):
    self.img_size = img_size
    self.loss = loss

    self.piano_roll_model = piano_roll_model
    self.CNNf = CNNf
    self.Q1 = Q1
    self.Q2 = Q2
    self.l1 = l1_l2
    self.l2 = l1_l2

    self.epochs = epochs
    self.batch_size = batch_size
    self.learning_rate=learning_rate
    self.validation_split = validation_split
    self.verbose = verbose
    self.drop_rate = droprate
    self.plot_loss = plot_loss

  def get_model(self, *_):

    # seed = 123
    # tf.random.set_seed(seed)
    # np.random.seed(seed)
    # tf.keras.backend.clear_session()
    regularizer_o = tf.keras.regularizers.OrthogonalRegularizer(factor=self.l2)

    inputA = tf.keras.layers.Input(shape=(img_size), name='entradaA')
    # convA = tf.keras.layers.Conv2D(self.CNNf, 5, strides=2, padding="same", name = 'conv_filter_A1',activation='selu')(inputA)
    # convA=  tf.keras.layers.MaxPooling2D(pool_size=2)(convA)
    # convA = tf.keras.layers.Conv2D(2*self.CNNf, 3, strides=2, padding="same", name = 'conv_filter_A2',activation='selu')(convA)
    # convA=  tf.keras.layers.MaxPooling2D(pool_size=2)(convA)
    # flattenA = tf.keras.layers.Flatten(input_shape=(convA.shape[1],convA.shape[2],convA.shape[3]))(convA)
    enco_pr = self.piano_roll_model.encoder(inputA)
    flattenA = layers.BatchNormalization()(enco_pr)
    h1A = layers.Dropout(self.drop_rate)(flattenA)
    h1A = tf.keras.layers.Dense(self.Q1,activation='selu',name='h1A',
                                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=self.l1,l2=self.l2))(h1A)
    h1A = layers.BatchNormalization()(h1A)
    h2A = tf.keras.layers.Dense(self.Q2,activation='linear',name='h2A',
                                kernel_regularizer=regularizer_o)(h1A)
    h2A = layers.BatchNormalization()(h2A)

    self.model = tf.keras.Model(inputs=[inputA],outputs=[h2A], name = 'cka_PianoRoll')
    return

  def fit(self, Xtrain, ytrain,*_):

    opt = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
    loss = self.loss

    callback1 = tf.keras.callbacks.TerminateOnNaN()

    self.get_model()

    self.model.compile(loss=loss, optimizer=opt)
    history = self.model.fit(x =[Xtrain], y=[ytrain],
                            epochs=self.epochs,batch_size=self.batch_size, # 32, 64, 128, 256
                             validation_split=self.validation_split,  callbacks=[callback1], verbose = self.verbose)


    return history

  def predict(self, X, *_):
    return self.model.predict(X)

  def plt_history(self):
    plt.plot(self.history.history['loss'])
    plt.plot(self.history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    return



In [23]:

# Define custom loss
#centered kernel alignmet
#http://www.jmlr.org/papers/volume13/cortes12a/cortes12a.pdf
#https://www.frontiersin.org/articles/10.3389/fnins.2017.00550/full
def custom_loss(scaleX=1,scaleY=1):
    #@tf.function()  #decorador para operar sobre python, mas lento y poco efectivo en muchos casos
    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
    def custom_cka_loss(y_true,y_pred): #ytrue labels, ypred  = Xw
        ####gradiente##########################################
        y_true = tf.cast(y_true,dtype=tf.float32)
        y_pred = tf.cast(y_pred,dtype=tf.float32)
        scalar_kernel = tfp.math.psd_kernels.ExponentiatedQuadratic(amplitude=1, length_scale= tf.cast(scaleX,dtype=tf.float32))
        scalar_kernely = tfp.math.psd_kernels.ExponentiatedQuadratic(amplitude=1, length_scale=tf.cast(scaleY,dtype=tf.float32))
        k = scalar_kernel.matrix(y_pred, y_pred)
        l = scalar_kernely.matrix(y_true, y_true)
        ######################################################
        N = tf.shape(l)[0]
        N2 = tf.cast(tf.shape(l)[0],dtype=tf.float32)
        h = tf.eye(N) - (1.0/N2)*tf.ones([N,1])*tf.ones([1,N]) #matrix for centered kernel
        trkl = tf.linalg.trace(tf.matmul(tf.matmul(k,h),tf.matmul(l,h)))
        trkk = tf.linalg.trace(tf.matmul(tf.matmul(k,h),tf.matmul(k,h)))
        trll = tf.linalg.trace(tf.matmul(tf.matmul(l,h),tf.matmul(l,h)))
        #####funcion de costo############################################3
        f     = -trkl/tf.sqrt(trkk*trll)#-tf.math.log(trkl)+0.5*tf.math.log(trkk)## negative cka cost function (minimizing) f \in [-1,0] #
        return f
    # Return a function
    return custom_cka_loss

In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm

def EEGNet_two_output(nb_classes, Chans = 64, Samples = 128,
             dropoutRate = 0.5, kernLength = 64, F1 = 8,
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:

        1. Depthwise Convolutions to learn spatial filters within a
        temporal convolution. The use of the depth_multiplier option maps
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn
        spatial filters within each filter in a filter-bank. This also limits
        the number of free parameters to fit when compared to a fully-connected
        convolution.

        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions.


    While the original paper used Dropout, we found that SpatialDropout2D
    sometimes produced slightly better results for classification of ERP
    signals. However, SpatialDropout2D significantly reduced performance
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.

    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the
    kernel lengths for double the sampling rate, etc). Note that we haven't
    tested the model performance with this rule so this may not work well.

    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D.
    Inputs:

      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D.
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
    """

    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')

    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   name='Conv2D_1',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False,
                                   name='Depth_wise_Conv2D_1',
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)

    block2       = SeparableConv2D(F2, (1, 16),
                                   name='Separable_Conv2D_1',
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)

    flatten      = Flatten(name = 'flatten')(block2)

    dense_label1        = Dense(nb_classes, name = 'output_valence', activation = "sigmoid",
                         kernel_constraint = max_norm(norm_rate),  )(flatten)
    dense_label2        = Dense(nb_classes, name = 'output_arousal', activation = "sigmoid",
                         kernel_constraint = max_norm(norm_rate))(flatten)
    #softmax      = Activation('softmax', name = 'out_activation')(dense)

    return Model(inputs=input1, outputs=[dense_label1, dense_label2])

# Training

In [25]:
#general Info
N_subs = 32
N_trials = 40
N_chann = 32
fs = 128 #Hz
len_data = 8064
labels = 0 #(valence, arousal, dominance, liking)
f_bank = np.asarray([4., 40.])
n_splits = 5

len_wind = 6 # 6 segs windows
N_windws =len_data/ (len_wind*fs) # 10 windows from 6 seconds



#train_test_data

Xpr_train = Prolls_array_train[:,24:88,:,np.newaxis].astype('float32').transpose(0,2,1,3)
ypr_train = Prolls_train_mask[:,24:88,:,np.newaxis].astype('float32').transpose(0,2,1,3)
Xpr_dims = Xpr_train.shape


print('EEG variables names: "X_train"' )
print('MIDI variables names: "Xpr_train"')

img_size = Xpr_dims[1::]
num_classes = 1
loss = DiceCoefficient()

#wide net parameters
Experiment = 'cross_val'
data_set = 'DEAP'
nunits = 1000
droprate = 0.4
l1 = 1e-4 #
l2 = l1
verbose = 0


#load paths
EEG_net_paths = '/content/drive/MyDrive/music_VAEs/DEAP_dataset/EEG/cross_val_results/'


#save paths
save_generated_midis = "/content/drive/MyDrive/music_VAEs/DEAP_dataset/match_networks/match_cross_val/generated_database/"
save_match_models_weigths = "/content/drive/MyDrive/music_VAEs/DEAP_dataset/match_networks/match_cross_val/match_net_models/"

EEG variables names: "X_train"
MIDI variables names: "Xpr_train"


In [26]:
path_DEAP = '/kaggle/input/deap-dataset'
save_path = '/content/drive/MyDrive/music_VAEs/DEAP_dataset/'
save_models_path = '/content/drive/MyDrive/music_VAEs/DEAP_dataset/EEG/cross_val_results/'


N_subs = 32
N_trials = 40
N_chann = 32
fs = 128 #Hz
len_data = 7680# len without the initial seconds 8064
len_wind = 6 # 6 segs windows
N_windws =len_data/ (len_wind*fs) # 10 windows from 6 seconds


labels = 1 #(valence, arousal, dominance, liking)
f_bank = np.asarray([4., 40.])
lr = 1e-2
epochs = 500
verbose = 0

seed = 123
folds = 5

##save data

data_set = 'DEAP'
model_name = 'multiclass'
Experiment = 'cross_val'

In [27]:
!mkdir /kaggle/working/all_windows_piano_roll_generated

In [28]:
from keras.utils import plot_model


In [30]:
for sub in range(32):

    print('Subject: '+ str (sub+1))
    #-------- load subject
    with open('/kaggle/input/deap-full/DEAP/data_preprocessed_python/s'+ f"{sub+1:02}" + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1')
    X = subject['data'][:,0:N_chann,int(3*fs):]

    ### target data
    y_valance = (subject['labels'][:,0] > 5)*1
    y_arousal = (subject['labels'][:,1] > 5)*1

    y = []
    for i in range(len(y_valance)):
      if y_valance[i] == 1 and y_arousal[i] == 1:
        y.append(0)
      elif y_valance[i] == 0 and y_arousal[i] == 1:
        y.append(1)
      elif y_valance[i] == 0 and y_arousal[i] == 0:
        y.append(2)
      elif y_valance[i] == 1 and y_arousal[i] == 0:
        y.append(3)

    y = np.array(y)


    Xdata = segmentation_trials(X, fs = fs, segs = len_wind ) # windowing 1 min trials in 6 seconds segments
    Xdata = Xdata[indx_full_audio,:,:,np.newaxis]

    #All train losses
    ###### multiclass
    y_data_multiclass = np.repeat(y, N_windws)[indx_full_audio] # reapiting layers 10 times
    ##### biclass
    y_data_valence = np.repeat(y_valance, N_windws)[indx_full_audio]
    y_data_arousal = np.repeat(y_arousal, N_windws)[indx_full_audio]

    Y_train = [y_data_valence, y_data_arousal]


    print(Xdata.shape, y_data_multiclass.shape)

    # not one hot
    y_valance = subject['labels'][:,0] / 10
    y_arousal = subject['labels'][:,1] / 10

    y_continuos_valence = np.repeat(y_valance, N_windws)[indx_full_audio]
    y_continuos_arousal = np.repeat(y_arousal, N_windws)[indx_full_audio]

    #--------- load Network

    reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=30, verbose=verbose, mode='min', min_delta=0.01, min_lr=0)
    terminate_on_nan = tf.keras.callbacks.TerminateOnNaN()
    callbacks = [reduce_lr_on_plateau, terminate_on_nan]
    #losses = [tf.keras.losses.BinaryCrossentropy(), tf.keras.losses.BinaryCrossentropy()]

    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
    for i, (train_index, test_index) in enumerate(sss.split(Xdata, y_data_valence)):
      print(f"fold  N°{i+1}")
      Xtrain_ = Xdata[train_index]
      Xtest_ = Xdata[test_index]

      y_train_valence = y_data_valence[train_index]
      y_test_valence = y_data_valence[test_index]

      y_train_arousal = y_data_arousal[train_index]
      y_test_arousal = y_data_arousal[test_index]


      ytrain_ = y_data_multiclass[train_index]
      ytest_ = y_data_multiclass[test_index]

      ### load pesos autoencoder

      img_size = Xpr_train.shape[-3::]
      loss = [DiceCoefficient(), custom_loss(scaleX=1,scaleY=0.5)]
      cka_pr = autoencoderCKA_Pianoroll(img_size, loss, Q2 = 2, validation_split=0.2, epochs=250, Dice_weigths = [0.7, 0.3])
      cka_pr.get_model()

      cka_pr.model.load_weights(f'/kaggle/input/midi-featuresandweigths/autoencoder_cka_weigths/ae_pr_sub{sub+1}.h5')

      ### load EEGnet
      losses = [tf.keras.losses.MeanSquaredError(), tf.keras.losses.MeanSquaredError()]

      opt = tf.keras.optimizers.Adam(learning_rate= 0.01)
      batch_size, Chans, Samples, _ = Xdata.shape
      model_args = {'nb_classes':1,'Chans':Chans,'Samples':Samples,'dropoutRate':0.6,
                  'kernLength':128,'F1':4,'D':4,'F2':32,'norm_rate':0.25,'dropoutType':'Dropout'}
      clf = EEGNet_two_output(nb_classes = model_args['nb_classes'],
                          Chans = model_args['Chans'],
                          Samples = model_args['Samples'],
                          dropoutRate = model_args['dropoutRate'],
                          kernLength = 128,
                          F1 = model_args['F1'], D = model_args['D'], F2 = model_args['F2'],
                          )
      clf.compile(loss= losses, optimizer=opt)
      # history = clf.fit( Xtrain_,[y_train_valence[:,np.newaxis], y_train_arousal[:,np.newaxis]], epochs = 500,
      #                   batch_size=500, callbacks=callbacks,
      #                    validation_split=0.2, verbose=1
      #                    )
    
      from keras.utils import plot_model
      from IPython.display import Image

      # Graficar el modelo del autoencoder CKA
      plot_model(cka_pr.model, to_file='cka_pr_model.png', show_shapes=True, show_layer_names=True)
      Image('cka_pr_model.png')

      # Graficar el modelo EEGNet
      plot_model(clf, to_file='eegnet_model.png', show_shapes=True, show_layer_names=True)
      Image('eegnet_model.png')

      history = clf.fit( Xtrain_,[y_continuos_valence[train_index,np.newaxis], y_continuos_arousal[train_index,np.newaxis]], epochs = 500,
                        batch_size=500, callbacks=callbacks,
                        validation_split=0.2, verbose=0
                        )

      y_pred_valence, y_pred_arousal=clf.predict(Xtest_)

      print(accuracy_score(np.where(y_pred_valence > 0.5, 1, 0 ), y_test_valence))
      print(accuracy_score(np.where(y_pred_arousal > 0.5, 1, 0 ), y_test_arousal))

      # generation

      y_pred_array = np.concatenate((y_pred_valence,y_pred_arousal), axis = 1)
      y_true_array = np.concatenate((y_continuos_valence[train_index,np.newaxis],y_continuos_arousal[train_index,np.newaxis]), axis = 1)
      # distance matrix

      y_dist = cdist(y_pred_array, y_true_array, 'euclidean')
    
      
      sigmoid_ydist   = tf.keras.layers.Softmax(axis=-1, name = 'softmax')(y_dist)
      #dense_ydist   = Dense(y_dist.shape[-1], name = 'softmax', activation = "softmax")(y_dist)

      # select top 5 nearest
      max_indices = np.argsort(-sigmoid_ydist.numpy(), axis=1)[:, :5]
      # Crear una matriz con 1 en los índices de los valores máximos y 0 en el resto
      result_array = np.zeros_like(sigmoid_ydist.numpy())
      rows = np.arange(sigmoid_ydist.shape[0])[:, np.newaxis]
      result_array[rows, max_indices] = 1
      #piano roll Generation
      Xpr_code= cka_pr.encoder(Xpr_train[train_index])
      print(sigmoid_ydist.shape, Xpr_code.shape)
      eeg_coding = np.matmul(result_array, Xpr_code)
      Xpr_pred = cka_pr.decoder(eeg_coding).numpy()
      #saving arrays
      np.save(f'/kaggle/working/all_windows_piano_roll_generated/sub{sub}_fold{i}', Xpr_pred)


Subject: 1


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/deap-full/DEAP/data_preprocessed_python/s01.dat'

# CONVERSION MIDI

In [ ]:
def rebuilPianoRolls(X):
  """
  this function takes an output array from the vae net and returns
  a piano roll array ready to be converted into MIDI file
  input
  X: array with dimensions [samples, time, pitch, 1]
  output
  proll_padding: array with dimension [sample x pitch x time ]
  """
  #trasnpose to [sample x time x pitch]
  proll = X[:,:,:,0].transpose(0,2,1)
  proll_padding =np.pad(proll, ((0,0), (24,40), (0, 0)), 'constant')
  return proll_padding

In [ ]:
!mkdir /kaggle/working/all_windows_MIDI_generated

In [ ]:
save_generated_midis = f"/kaggle/working/all_windows_MIDI_generated"

for sub in range(32):

  print(f'saving MIDI sub: {sub}')
  for fold in range(5):
    midiGEnerated = np.load(f'/kaggle/working/all_windows_piano_roll_generated/sub{sub}_fold{fold}.npy')
    midiGEnerated = rebuilPianoRolls(midiGEnerated)
    print(f'{"saving MIDI generated":-^50}')
     # --- create directoris to save MIDI ------
    midi_save_path = os.path.join(save_generated_midis, 'MIDI_sub_'+str(sub+1))
    try:
        os.mkdir(midi_save_path)
    except OSError as error:
        print(error)
    #----- saving data --------
    for sample in range(midiGEnerated.shape[0]):
      proll = np.where((midiGEnerated[sample])>0.1,100,0)
      mid_new = piano_roll_to_pretty_midi(proll, fs=fProll)
      mid_new.write(midi_save_path+'/mid_fold'+str(fold)+'_sub_'+str(sample)+'.mid')
    print('--------- MIDI data saved---')
    print(f'{"MIDI data saved":-^50}')



In [ ]:
midiGEnerated = np.load(f'/kaggle/working/all_windows_piano_roll_generated/sub{11}_fold{4}.npy')

for sample in range(10):

  # Datos de predicción
  result_array_prediccion = midiGEnerated[sample,:,:,0]

  result_array_prediccion2 = midiGEnerated[sample*2,:,:,0]

  plt.figure(figsize=(9, 2))
  # Primer subplot para los datos originales
  plt.subplot(1, 2, 1)
  plt.pcolormesh(np.where(result_array_prediccion.T > 0.01, 1, 0))
  plt.colorbar()
  plt.title("Original")
  # Segundo subplot para los datos de predicción
  plt.subplot(1, 2, 2)
  plt.pcolormesh(np.where(result_array_prediccion2.T > 0.01, 1, 0))
  plt.colorbar()
  plt.title("Prediction")
  # Ajustar el espaciado entre subplots
  plt.tight_layout()
  # Mostrar la figura con los dos subplots
  plt.show()